In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import _pickle as pickle
import random
import scipy.stats as ss

# cost features

In [2]:
startTime = datetime.now()

input_age_seq = pickle.load(open("../data/baseline/input_age_seq","rb"))
input_sex_seq = pickle.load(open("../data/baseline/input_sex_seq","rb"))

input_cost_seq = pickle.load(open("../data/baseline/input_cost_seq","rb"))
target_cost_seq = pickle.load(open("../data/baseline/target_cost_seq","rb"))

input_medical_cost_seq = pickle.load(open("../data/baseline/input_medical_cost_seq","rb"))
input_monthly_medical_cost_seq = pickle.load(open("../data/baseline/input_monthly_medical_cost_seq","rb"))

input_pharmacy_cost_seq = pickle.load(open("../data/baseline/input_pharmacy_cost_seq","rb"))
input_monthly_pharmacy_cost_seq = pickle.load(open("../data/baseline/input_monthly_pharmacy_cost_seq","rb"))

print(datetime.now() - startTime)

0:00:01.349851


In [3]:
def build_feature(seq, size):    
    X = np.zeros((len(seq), size))
    for i in range(len(seq)):
        value = seq[i]
        X[i][value] = 1
    return X

In [4]:
age_feature = build_feature([i//5 for i in input_age_seq], 4)
sex_feature = build_feature([1 if i=="M" else 0 for i in input_sex_seq], 2)

cost_feature = np.array([np.log(i+1) for i in input_cost_seq]).reshape(-1,1)
medical_cost_feature = np.array([np.log(i+1) for i in input_medical_cost_seq]).reshape(-1,1)
pharmacy_cost_feature = np.array([np.log(i+1) for i in input_pharmacy_cost_seq]).reshape(-1,1)

monthly_medical_cost_feature = np.array([[np.log(i+1) for i in y] for y in input_monthly_medical_cost_seq]).reshape(-1,12)
monthly_pharmacy_cost_feature = np.array([[np.log(i+1) for i in y] for y in input_monthly_pharmacy_cost_seq]).reshape(-1,12)

In [5]:
features = [age_feature, sex_feature, cost_feature, medical_cost_feature, pharmacy_cost_feature]

X_cost = np.concatenate(features, axis =1)
X_cost.shape

(143102, 9)

In [6]:
y = np.array([np.log(i+1) for i in target_cost_seq])
# y = np.array([x/len(target_cost_seq) for x in ss.rankdata(target_cost_seq)])

# util sequence

In [7]:
startTime = datetime.now()

input_util_seq = pickle.load(open("../data/advance/input_util_seq","rb"))

print(datetime.now() - startTime)

0:00:06.072725


In [8]:
vocab = {}
for p in input_util_seq:
    for v in p:
        for c in v:
            if c not in vocab: vocab[c] = len(vocab)

In [9]:
def build_seq_feature(seq, vocab):
    X = np.zeros((len(seq),12, len(vocab) ))
    for i in range(len(seq)):
        for j in range(12):
            for value in seq[i][j]:
                X[i][j][vocab[value]] +=1
    return X

In [10]:
X_util = build_seq_feature(input_util_seq, vocab)

In [11]:
X_util = np.concatenate((monthly_medical_cost_feature.reshape(-1,12,1),\
                         monthly_pharmacy_cost_feature.reshape(-1,12,1),\
                         X_util), axis=-1)

In [12]:
X_util.shape

(143102, 12, 38)

# code sequence

In [13]:
startTime = datetime.now()

input_diag_seq = pickle.load(open("../data/advance/input_diag_seq","rb"))
input_proc_seq = pickle.load(open("../data/advance/input_proc_seq","rb"))
input_drug_seq = pickle.load(open("../data/advance/input_drug_seq","rb"))

print(datetime.now() - startTime)

0:00:11.213560


In [14]:
code2int = {"PAD":0}

code_seq = []
for p_diag, p_proc, p_drug in zip(input_diag_seq, input_proc_seq, input_drug_seq):
    new_p = []
    for diag, proc, drug in zip(p_diag, p_proc, p_drug):
        new_v = []
        for d in diag:
            if d not in code2int: code2int[d] = len(code2int)
            new_v.append(code2int[d])
        for p in proc:
            if p not in code2int: code2int[p] = len(code2int)
            new_v.append(code2int[p])
        for dr in drug:
            if dr not in code2int: code2int[dr] = len(code2int)
            new_v.append(code2int[dr])
        new_p.append(new_v)
    code_seq.append(new_p)
    
len(code2int)

10918

In [15]:
def build_seq(seq, max_codes = 50, max_length=12):
    X = np.zeros((len(seq), max_length, max_codes))
    for i, p in enumerate(seq):
        assert len(p) == max_length
        for j, claim in enumerate(p):
            claim = claim[:max_codes]
            X[i][j][:len(claim)] = claim
    return X

In [16]:
X_code = build_seq(code_seq)
X_code.shape

(143102, 12, 50)

# Model

In [17]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import scipy

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K

from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer

In [19]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(name='{}_W'.format(self.name),
                                 shape=(input_shape[-1],),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(name='{}_b'.format(self.name),
                                     shape=(input_shape[1],),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c, a

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim
    

In [23]:
def build_model(max_len =12,
                max_code=50,
                max_util=38,
                max_demo=9,
                feature_code=len(code2int),
                embed_dim = 50,
                lstm_units=32,
               ):
    
    input_code = layers.Input(shape=(max_len, max_code))
    input_util = layers.Input(shape=(max_len, max_util))
    input_demo = layers.Input(shape=(max_demo,))
    inputs_list = [input_code, input_util, input_demo]
    
    # code
    tmp_input = layers.Input(shape=(max_code, ))
    tmp = layers.Embedding(input_dim=feature_code, output_dim=embed_dim, 
                           mask_zero=True, name='code_embedding')(tmp_input)
    
    tmp = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(tmp)
    tmp, ait = Attention(max_code)(tmp)
    codeEncoder = keras.models.Model(tmp_input, tmp)    
    
    input_code = layers.TimeDistributed(codeEncoder)(input_code)
    input_code = layers.Bidirectional(layers.GRU(lstm_units, return_sequences=True))(input_code)
    input_code, ait2 = Attention(max_len)(input_code)
        
    # util
    input_util = layers.Dense(lstm_units, activation="relu")(input_util)
    
    input_util = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(input_util)
    input_util, ait3 = Attention(max_len)(input_util)
    
    # demo
    out = layers.concatenate([input_code, input_util, input_demo])
    
    out = layers.Dense(lstm_units, activation="relu")(out)
    out = layers.Dropout(0.5)(out)

    out = layers.Dense(lstm_units, activation="relu")(out)
    out = layers.Dropout(0.5)(out)
    
    out = layers.Dense(1, activation=None, name='main_output')(out)
    model = keras.models.Model(inputs=inputs_list, outputs=[out])

    model.compile(optimizer='adam', loss="mse")
#     print(model.summary())
#     print(codeEncoder.summary())

    return model

In [26]:
def result(y_true, y_pred):
    return metrics.mean_absolute_error(y_true, y_pred), \
           metrics.r2_score(y_true, y_pred),\
           np.sqrt(metrics.mean_squared_error(y_true, y_pred)),\
           scipy.stats.pearsonr(y_true, y_pred)[0]

def generate_result(seed):
    model = build_model()
    idx_train, idx_val = train_test_split(range(len(y)), train_size=0.85, random_state=seed)
    idx_train, idx_test = train_test_split(range(len(idx_train)), train_size=0.82, random_state=seed)

    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='min', restore_best_weights=True)
    history = model.fit([X_code[idx_train], X_util[idx_train], X_cost[idx_train]], y[idx_train], epochs=50, batch_size=100, \
                        validation_data=([X_code[idx_val], X_util[idx_val], X_cost[idx_val]], y[idx_val]), verbose=1, callbacks=[earlyStopping])


    y_pred = model.predict([X_code[idx_test], X_util[idx_test], X_cost[idx_test]], verbose=0).reshape(-1)
    mae, r2, rmse, pcc = result(y[idx_test], y_pred)
    return mae, r2, rmse, pcc, y_pred

def display(list_eva):
    for list_ in list_eva:
        print(np.mean(list_), np.std(list_))
        print()

In [27]:
mae_list, r2_list, rmse_list, pcc_list = [], [], [], []

for i in range(5):
    print(i)
    mae, r2, rmse, pcc, y_pred = generate_result(seed=i)
    mae_list.append(mae)
    r2_list.append(r2)
    rmse_list.append(rmse)
    pcc_list.append(pcc)
    
display([mae_list, r2_list, rmse_list, pcc_list])

0
Epoch 1/50
998/998 [==============================] - 702s 703ms/step - loss: 6.4518 - val_loss: 2.7454
Epoch 2/50
998/998 [==============================] - 664s 666ms/step - loss: 4.3538 - val_loss: 2.7529
Epoch 3/50
998/998 [==============================] - 644s 645ms/step - loss: 3.8242 - val_loss: 2.6899
Epoch 4/50
998/998 [==============================] - 644s 645ms/step - loss: 3.5557 - val_loss: 2.8653
Epoch 5/50
998/998 [==============================] - 643s 645ms/step - loss: 3.2931 - val_loss: 2.5707
Epoch 6/50
998/998 [==============================] - 640s 641ms/step - loss: 3.0969 - val_loss: 2.5961
Epoch 7/50
998/998 [==============================] - 642s 643ms/step - loss: 2.9190 - val_loss: 2.5557
Epoch 8/50
998/998 [==============================] - 644s 645ms/step - loss: 2.7916 - val_loss: 2.5360
Epoch 9/50
998/998 [==============================] - 641s 642ms/step - loss: 2.6665 - val_loss: 2.5526
Epoch 10/50
998/998 [==============================] - 645s 64